# DQX

In [0]:
https://adb-5943863453538272.12.azuredatabricks.net/editor/notebooks/4441860942440814?o=5943863453538272

In [0]:
databricks labs dqx open-dashboards

## Reference

### DQX Reference

#### DQEngine Methods

| Method                                 | Purpose                                                        | Local?    | Notes                                                   |
|-----------------------------------------|----------------------------------------------------------------|-----------|---------------------------------------------------------|
| apply_checks                           | Apply DQRule checks to DataFrame, adds _warning/_error cols    | Yes       | Use for reporting-only; does not split DF               |
| apply_checks_and_split                  | Apply checks, returns valid/quarantine DFs                     | Yes       | Use if you want to split valid/invalid rows             |
| apply_checks_and_save_in_table          | Apply checks, save valid/quarantine DFs to Delta tables        | No        | Needs OutputConfig and QuarantineConfig                 |
| apply_checks_by_metadata                | Apply YAML/Delta table checks to DF, adds _warning/_error      | Yes       | Ideal for rules from YAML/Delta; no split               |
| apply_checks_by_metadata_and_split      | Same, but returns valid/quarantine DFs                         | Yes       | Only use if you want to split output                    |
| apply_checks_by_metadata_and_save_in_table | Metadata checks, save valid/quarantine to Delta tables        | No        | Needs OutputConfig and QuarantineConfig                 |
| validate_checks                        | Validates structure/types of check config                      | Yes       | Use in CI/CD for YAML validation                        |
| get_invalid                            | Extract rows with warnings/errors                              | Yes       |                                                        |
| get_valid                              | Extract only valid rows                                        | Yes       |                                                        |
| load_checks_from_local_file             | Read checks from YAML/JSON file                                | Yes       | For CI/CD/local dev                                     |
| save_checks_in_local_file               | Write checks to YAML/JSON                                      | Yes       |                                                        |
| load_checks_from_workspace_file         | Load checks from Databricks workspace file                     | No        | Only in DBX context                                     |
| save_checks_in_workspace_file           | Save checks to workspace file                                  | No        |                                                        |
| load_checks_from_installation           | Load checks from install dir                                   | No        |                                                        |
| save_checks_in_installation             | Save checks to install dir                                     | No        |                                                        |
| save_results_in_table                   | Save (output_df, quarantine_df) to Delta tables                | No        | Needs OutputConfig (and optional QuarantineConfig)      |
| load_run_config                        | Load run config from install dir                               | No        |                                                        |

#### Pre-defined Functions

In [0]:
ROW_LEVEL_FUNCTIONS = {
    "is_not_null": {
        "args": ["column"],
        "optional": []
    },
    "is_not_empty": {
        "args": ["column"],
        "optional": []
    },
    "is_not_null_and_not_empty": {
        "args": ["column"],
        "optional": ["trim_strings"]
    },
    "is_in_list": {
        "args": ["column", "allowed"],
        "optional": []
    },
    "is_not_null_and_is_in_list": {
        "args": ["column", "allowed"],
        "optional": []
    },
    "is_not_null_and_not_empty_array": {
        "args": ["column"],
        "optional": []
    },
    "is_in_range": {
        "args": ["column", "min_limit", "max_limit"],
        "optional": []
    },
    "is_not_in_range": {
        "args": ["column", "min_limit", "max_limit"],
        "optional": []
    },
    "is_not_less_than": {
        "args": ["column", "limit"],
        "optional": []
    },
    "is_not_greater_than": {
        "args": ["column", "limit"],
        "optional": []
    },
    "is_valid_date": {
        "args": ["column"],
        "optional": ["date_format"]
    },
    "is_valid_timestamp": {
        "args": ["column"],
        "optional": ["timestamp_format"]
    },
    "is_not_in_future": {
        "args": ["column", "offset"],
        "optional": ["curr_timestamp"]
    },
    "is_not_in_near_future": {
        "args": ["column", "offset"],
        "optional": ["curr_timestamp"]
    },
    "is_older_than_n_days": {
        "args": ["column", "days"],
        "optional": ["curr_date", "negate"]
    },
    "is_older_than_col2_for_n_days": {
        "args": ["column1", "column2", "days"],
        "optional": ["negate"]
    },
    "regex_match": {
        "args": ["column", "regex"],
        "optional": ["negate"]
    },
    "is_valid_ipv4_address": {
        "args": ["column"],
        "optional": []
    },
    "is_ipv4_address_in_cidr": {
        "args": ["column", "cidr_block"],
        "optional": []
    },
    "sql_expression": {
        "args": ["expression"],
        "optional": ["msg", "name", "negate", "columns"]
    }
}

In [0]:
DATASET_LEVEL_FUNCTIONS = {
    "is_unique": {
        "args": ["columns"],
        "optional": ["nulls_distinct"],
    },
    "is_aggr_not_greater_than": {
        "args": ["limit"],
        "optional": ["column", "aggr_type", "group_by"],
    },
    "is_aggr_not_less_than": {
        "args": ["limit"],
        "optional": ["column", "aggr_type", "group_by"],
    },
    "is_aggr_equal": {
        "args": ["limit"],
        "optional": ["column", "aggr_type", "group_by"],
    },
    "is_aggr_not_equal": {
        "args": ["limit"],
        "optional": ["column", "aggr_type", "group_by"],
    },
    "foreign_key": {
        "args": ["columns", "ref_columns"],
        "optional": ["ref_df_name", "ref_table", "negate"],
    },
    "sql_query": {
        "args": ["query", "merge_columns", "condition_column"],
        "optional": ["input_placeholder", "msg", "name", "negate"],
    },
    "compare_datasets": {
        "args": ["columns", "ref_columns"],
        "optional": ["exclude_columns", "ref_df_name", "ref_table", "check_missing_records"],
    },
}

####  [Output Column Structure](https://databrickslabs.github.io/dqx/docs/guide/quality_checks/#quality-check-results)

| **Field**         | **Description**                                                                 |
|-------------------|---------------------------------------------------------------------------------|
| `name`            | Name of the check (string type).                                                |
| `message`         | Message describing the quality issue (string type).                             |
| `columns`         | Name of the column(s) where the quality issue was found (string type).          |
| `filter`          | Filter applied if any (string type).                                            |
| `function`        | Rule/check function applied (string type).                                      |
| `run_time`        | Timestamp when the check was executed (timestamp type).                         |
| `user_metadata`   | Optional key-value custom metadata provided by the user (dictionary type).      |

In [0]:
from pyspark.sql.types import StructType, StructField, ArrayType, StringType, TimestampType, MapType

dq_result_item_schema = StructType(
    [
        StructField("name", StringType(), nullable=True),
        StructField("message", StringType(), nullable=True),
        StructField("columns", ArrayType(StringType()), nullable=True),
        StructField("filter", StringType(), nullable=True),
        StructField("function", StringType(), nullable=True),
        StructField("run_time", TimestampType(), nullable=True),
        StructField("user_metadata", MapType(StringType(), StringType()), nullable=True),
    ]
)

dq_result_schema = ArrayType(dq_result_item_schema)

In [0]:
[
  {
    "name": "col_city_is_null",
    "message": "Column 'city' is null",
    "columns": ["city"],
    "filter": "country = 'Poland'",
    "function": "is_not_null",
    "run_time": "2025-01-01 14:31:21",
    "user_metadata": {"key1": "value1", "key2": "value2"},
  },
]

## Custom Rules

### Examples

In [0]:
################################################################################
# Section 1 – Simple row‑level checks
#
# These rules operate on a single column at a time for each row.  Only the
# `column` argument is mandatory.  Additional arguments apply only to certain
# functions and are shown commented out with explanations.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error                     # 'error' => quarantine row on failure; 'warn' => flag but keep row
  run_config_name: default               # added run configuration name
  check:
    function: is_not_null                # any row-level built-in function
    arguments:
      column: col1                       # name or expression of a column in the table being checked
      # allowed: [1, 2, 3]               # for is_in_list / is_not_null_and_is_in_list: values the column must belong to
      # min_limit: 0                     # for is_in_range / is_not_in_range: lower bound (inclusive)
      # max_limit: 10                    # for is_in_range / is_not_in_range: upper bound (inclusive)
      # limit: 5                         # for is_not_less_than / is_not_greater_than: numeric/date/timestamp limit
      # date_format: yyyy-MM-dd          # for is_valid_date: expected date pattern
      # timestamp_format: yyyy-MM-dd HH:mm:ss   # for is_valid_timestamp: expected timestamp pattern
      # offset: 86400                    # for is_not_in_future / is_not_in_near_future: offset in seconds
      # curr_timestamp: '2025-08-05T00:00:00'   # override current timestamp; string or timestamp literal
      # days: 7                          # is_older_than_n_days / is_older_than_col2_for_n_days
      # column2: other_col               # required for is_older_than_col2_for_n_days
      # regex: '[A-Z]{3}[0-9]{4}'        # regex_match
      # negate: false                    # set true to invert the match
      # cidr_block: '192.168.1.0/24'     # is_ipv4_address_in_cidr
      # trim_strings: true               # is_not_null_and_not_empty

################################################################################
# Section 2 – Row‑level checks applied to multiple columns individually
#
# Use `for_each_column` when the same single‑column rule should be executed on
# several columns.  Each entry must be a valid column name or expression from
# the table being checked.  There is no hard limit on the number of columns.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_not_null
    for_each_column:
      - col1                             # first column name in the dataset
      - col2                             # second column name in the dataset
      # - col3                          # add additional column names as needed; each must exist in the dataset

################################################################################
# Section 3 – Row‑level checks on complex types (struct, map, array)
#
# These templates target elements within complex columns.  Use dot notation
# (`struct_field`) for struct fields, and `try_element_at` for map or array
# elements.  For array aggregations (e.g. max/min), include a numeric limit.
################################################################################

# Single element from a complex type
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_not_null
    arguments:
      column: col8.field1                # struct field; must refer to an existing field
      # column: try_element_at(col7, 'key1')  # map element lookup; key must exist
      # column: try_element_at(col4, 0)       # array element lookup by zero-based index
      # Any optional parameters from Section 1 can be included here (e.g. regex)

# Aggregating an array (e.g. ensure max element ≤ 10)
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_not_greater_than        # or is_not_less_than
    arguments:
      column: array_max(col4)            # aggregate function applied to an array column
      limit: 10                          # required for array aggregation; numeric threshold for comparison

################################################################################
# Section 4 – Row‑level SQL expression
#
# For complex row‑level logic, use `sql_expression`.  This rule does not take a
# `column` field; instead provide an SQL predicate.  Optional fields allow you to
# customise messages, naming and inversion behaviour.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: sql_expression
    arguments:
      expression: "col3 >= col2 AND col3 <= 10"  # SQL predicate; fails when expression is True
      # msg: "col3 out of range"         # optional message to include in result rows
      # name: "col3_range_check"         # optional name for the resulting check column (appears in result DataFrame)
      # negate: false                    # boolean: set true to invert the logic (fail when expression is False)
      # columns: [col2, col3]            # optional list of one or more column names used only for reporting; does not affect logic

################################################################################
# Section 5 – Dataset‑level uniqueness check
#
# Ensures that the specified columns are unique across all rows.  A failure is
# logged for every duplicate row found.  Use `nulls_distinct` to control how
# NULL values are treated.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_unique
    arguments:
      columns:
        - col1                            # key columns (one or more) used to test uniqueness
        # - col2                         # add additional columns for composite key
      # nulls_distinct: true              # default true: treat NULLs as distinct; set false to consider NULLs equal

################################################################################
# Section 6 – Dataset‑level aggregation checks
#
# Compare aggregated values against a threshold.  Specify `aggr_type` (count,
# sum, avg, min, max).  For count, omit the column; otherwise provide the
# column being aggregated.  `group_by` can list one or more columns to group
# rows before aggregation.
################################################################################

# Count of all rows must not exceed 10
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_aggr_not_greater_than
    arguments:
      aggr_type: count                   # aggregation type (count, sum, avg, min, max)
      limit: 10                          # numeric or literal threshold for comparison

# Count of non-null values in col2 must not exceed 10
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_aggr_not_greater_than
    arguments:
      column: col2                       # column to aggregate (omit for count of all rows)
      aggr_type: count
      limit: 10

# Count of col2 grouped by col3 must not exceed 10
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_aggr_not_greater_than
    arguments:
      column: col2
      aggr_type: count
      group_by:
        - col3                          # optional grouping; one or more columns; must exist in the table
      limit: 10

################################################################################
# Section 7 – Dataset‑level foreign key check
#
# Validates that values in the source dataset exist in a reference dataset.
# Provide either a reference DataFrame name (`ref_df_name`) or a fully qualified
# table name (`ref_table`), not both.  `negate` flips the logic to flag rows
# that do exist in the reference.
################################################################################

# Single-column foreign key using a reference DataFrame
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: foreign_key
    arguments:
      columns:
        - col1                           # column(s) in source dataset
      ref_columns:
        - ref_col1                       # matching column(s) in reference dataset
      ref_df_name: ref_df_key            # key to locate the reference DataFrame in ref_dfs
      # negate: false                    # optional; set true to fail rows that exist in the reference

# Composite-key foreign key using a table
- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: foreign_key
    arguments:
      columns:
        - col1
        - col2
      ref_columns:
        - ref_col1
        - ref_col2
      ref_table: catalog.schema.ref_table   # fully qualified table name used as the reference

################################################################################
# Section 8 – Dataset‑level SQL query
#
# Executes an arbitrary SQL query across the dataset (and optional reference data).
# The query must return all `merge_columns` plus a boolean `condition_column`.
# The check fails when the condition column is True (unless `negate` is true).
# Use `input_placeholder` to name the input DataFrame within the SQL query.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: sql_query
    arguments:
      query: |
        SELECT col1, col2, SUM(col3) = 0 AS condition
        FROM {{ input_view }}                  # placeholder referencing the input dataset
        GROUP BY col1, col2
      merge_columns:                           # must exist in input DataFrame and query result
        - col1
        - col2
      condition_column: condition              # boolean; True means failure
      input_placeholder: input_view            # alias used inside double braces in the SQL
      # msg: "Aggregated col3 is zero"       # optional message added to result rows
      # name: "check_sum_col3_zero"         # optional name of the resulting check column
      # negate: false                        # optional; set true to invert the meaning (fail when condition is False)

################################################################################
# Section 9 – Dataset‑level compare datasets
#
# Compares two datasets at row and column level.  Provide matching key columns
# for both source and reference.  You can exclude columns from the comparison
# and enable detection of missing records via full outer join.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: compare_datasets
    arguments:
      columns:
        - col1                           # key columns in the source dataset
        - col2
      ref_columns:
        - ref_col1                       # corresponding key columns in the reference dataset
        - ref_col2
      # ref_df_name: ref_df_key         # OR use ref_table: catalog.schema.ref_table
      # exclude_columns:
      #   - col7                        # columns to ignore differences for
      # check_missing_records: true     # if true, perform FULL OUTER JOIN to detect missing rows; increases result size

################################################################################
# Section 10 – Dataset‑level multi‑key application (`for_each_column`)
#
# Use this construct for functions that accept a `columns` argument (e.g.
# `is_unique`).  Each inner list defines a distinct key on which the check will
# run independently.
################################################################################

- table_name: catalog.schema.table       # added table name placeholder
  criticality: error
  run_config_name: default               # added run configuration name
  check:
    function: is_unique
    for_each_column:
      - [col1]                           # first uniqueness key: single column in the dataset; run uniqueness check on col1 alone
      - [col2, col3]                     # second key: composite of col2 and col3; run uniqueness check on (col2, col3)
      # - [col4, col5, col6]             # additional keys; each inner list must contain existing column names

################################################################################
# End of template with placeholders
################################################################################


In [0]:
# apply check to multiple columns (simple col, struct, map and array)
- table_name: catalog.schema.table       # added table name placeholder
  name: rule_name
  criticality: error
  check:
    function: is_not_null
    for_each_column:
    - col1 # col
    - col8.field1 # struct col
    - try_element_at(col7, 'key1') # map col
    - try_element_at(col4, 1) # array col 

| employee_id | age |  salary | hire_date  |
|------------:|----:|--------:|-----------|
| 1           | 30 |  50000  | 2020-01-15 |
| 2           | 45 |  80000  | 2018-05-30 |
| 3           | 25 |  45000  | 2022-07-10 |
| 4           | 40 |  70000  | 2019-09-20 |
| 5           | 29 |      0  | 2021-03-12 |
| 6           | 60 | 120000  | 2010-11-05 |

In [0]:
# Section 1 – Simple row-level check
# Ensure salary is within a reasonable range [0, 200000].
- table_name: dq_dev.company.employees
  name: salary_range
  criticality: error
  run_config_name: default
  check:
    function: is_in_range
    arguments:
      column: salary
      min_limit: 0
      max_limit: 200000

# Section 2 – Row-level check applied individually
# Ensure both age and salary are at least 1.
- table_name: dq_dev.company.employees
  name: age_salary_min
  criticality: warn
  run_config_name: default
  check:
    function: is_not_less_than
    for_each_column:
      - age
      - salary
    arguments:
      limit: 1

# Section 3 – Complex types
# (a) Validate that hire_date is a valid date (format yyyy-MM-dd).
- table_name: dq_dev.company.employees
  name: valid_hire_date
  criticality: error
  run_config_name: default
  check:
    function: is_valid_date
    arguments:
      column: hire_date
      date_format: yyyy-MM-dd

# (b) Ensure salary is all numeric digits.
- table_name: dq_dev.company.employees
  name: salary_numeric
  criticality: warn
  run_config_name: default
  check:
    function: regex_match
    arguments:
      column: salary
      regex: '^[0-9]+$'
      negate: false

# (c) Ensure employee_id is present.
- table_name: dq_dev.company.employees
  name: employee_id_exists
  criticality: error
  run_config_name: default
  check:
    function: is_not_null
    arguments:
      column: employee_id

# (d) Ensure salary does not exceed 200000.
- table_name: dq_dev.company.employees
  name: max_salary_le_200k
  criticality: warn
  run_config_name: default
  check:
    function: is_not_greater_than
    arguments:
      column: salary
      limit: 200000

# Section 4 – Row-level SQL expression
# Ensure salary is at least 1000 times age.
- table_name: dq_dev.company.employees
  name: salary_vs_age_rule
  criticality: error
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "salary >= age * 1000"
      msg: "Salary is unexpectedly low for age"
      name: "salary_vs_age"
      columns: [salary, age]

# Section 5 – Dataset-level uniqueness
# Ensure employee_id is unique.
- table_name: dq_dev.company.employees
  name: unique_employee_id
  criticality: error
  run_config_name: default
  check:
    function: is_unique
    arguments:
      columns: [employee_id]
      nulls_distinct: false

# Section 6 – Dataset-level aggregation
# Ensure the average salary is at least 30000.
- table_name: dq_dev.company.employees
  name: avg_salary_min
  criticality: warn
  run_config_name: default
  check:
    function: is_aggr_not_less_than
    arguments:
      column: salary
      aggr_type: avg
      limit: 30000

# Section 7 – Foreign key
# Ensure employee_id exists in a master employee table.
- table_name: dq_dev.company.employees
  name: employee_fk_check
  criticality: error
  run_config_name: default
  check:
    function: foreign_key
    arguments:
      columns: [employee_id]
      ref_columns: [employee_id]
      ref_table: catalog.hr.employees_master
      negate: false

# Section 8 – Dataset-level SQL query
# Ensure there are at least two employees in each age group.
- table_name: dq_dev.company.employees
  name: age_group_count_check
  criticality: error
  run_config_name: default
  check:
    function: sql_query
    arguments:
      query: |
        SELECT age,
               COUNT(*) < 2 AS condition      -- condition = True when there are < 2 employees in the group
        FROM {{ input_view }}
        GROUP BY age
      merge_columns: [age]
      condition_column: condition
      input_placeholder: input_view
      msg: "Age group has fewer than two employees"
      name: "min_two_employees_per_age"

# Section 9 – Compare datasets
# Compare current employees against last month's snapshot by employee_id, ignoring salary.
- table_name: dq_dev.company.employees
  name: compare_employees
  criticality: warn
  run_config_name: default
  check:
    function: compare_datasets
    arguments:
      columns: [employee_id]
      ref_columns: [employee_id]
      ref_df_name: last_month                 # Reference DataFrame name
      exclude_columns: [salary]               # Ignore salary differences
      check_missing_records: true             # Identify missing or new employee_ids

# Section 10 – Multi-key uniqueness
# Ensure both employee_id and (age, salary) keys are unique.
- table_name: dq_dev.company.employees
  name: multi_key_uniqueness
  criticality: error
  run_config_name: default
  check:
    function: is_unique
    for_each_column:
      - [employee_id]                          # First key
      - [age, salary]                          # Second key (composite)

### CLA Rules

#### wkdy_dim_project.yaml

In [0]:
#Dataset level
- check:
  arguments:
    columns: project_key
  function: is_unique
  criticality: error
name: project_key_is_not_unique
#row level
- check:
    arguments:
      column: project_key
    function: is_not_null
  criticality: error
  name: project_key_is_null
  
- check:
    arguments:
      column: project_status
    function: is_not_null
  criticality: error
  name: project_status_is_null
  
  - check:
    arguments:
      allowed:
        - Active
        - Closed
        - Pending Close
        - Schedule Pending
        - Suspended
      column: project_status
    function: is_in_list
  criticality: warning
  name: project_status_is_new_value
  
 # project type is not null or Unknown
 - check:
    arguments:
      expression: project_type_name is not null AND project_type_name != 'Unknown'
    function: sql_expression
  criticality: warning
  name: project_type_is_not_null_or_unknown

In [0]:
# Enforce uniqueness of project_key across the table (dataset-level)
- table_name: de_prd.gold.wkdy_dim_project
  name: project_key_is_not_unique
  criticality: warn
  run_config_name: default
  check:
    function: is_unique
    arguments:
      columns: [project_key]

# project_key must not be null (row-level)
- table_name: de_prd.gold.wkdy_dim_project
  name: project_key_is_null
  criticality: warn
  run_config_name: default
  check:
    function: is_not_null
    arguments:
      column: project_key

# project_status must not be null (row-level)
- table_name: de_prd.gold.wkdy_dim_project
  name: project_status_is_null
  criticality: warn
  run_config_name: default
  check:
    function: is_not_null
    arguments:
      column: project_status

# project_status must be from allowed set (row-level)
- table_name: de_prd.gold.wkdy_dim_project
  name: project_status_is_new_value
  criticality: warn
  run_config_name: default
  check:
    function: is_in_list
    arguments:
      column: project_status
      allowed:
        - Active
        - Closed
        - Pending Close
        - Schedule Pending
        - Suspended

# project_type_name must not be null or 'Unknown'
- table_name: de_prd.gold.wkdy_dim_project
  name: project_type_is_not_null_or_unknown
  criticality: warn
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "project_type_name is not null AND project_type_name != 'Unknown'"

# email should match pattern (basic email format)
- table_name: de_prd.gold.wkdy_dim_project
  name: email_is_not_valid
  criticality: warn
  run_config_name: default
  check:
    function: regex_match
    arguments:
      column: email
      regex: "^(.+)@(.+)$"

# project_start_date must be <= project_end_date
- table_name: de_prd.gold.wkdy_dim_project
  name: project_start_after_end_date
  criticality: warn
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(project_start_date, '1900-01-01') <= coalesce(project_end_date, '9999-12-31')"

# first_activity_date must be <= last_activity_date
- table_name: de_prd.gold.wkdy_dim_project
  name: first_activity_date_after_last_activity_date
  criticality: warn
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(first_activity_date, '1900-01-01') <= coalesce(last_activity_date, '9999-12-31')"

# _created_date must be <= _last_updated_date
- table_name: de_prd.gold.wkdy_dim_project
  name: created_date_after_last_updated_date
  criticality: warn
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(_created_date, '1900-01-01') <= coalesce(_last_updated_date, '9999-12-31')"

## DQX Intern Demo

In [0]:
# Red background + logo + title (wider and taller)
RED   = "#FF0000"
WHITE = "#FFFFFF"
title = "DQX Demo"

# Load the logo (no SVG fallback)
import base64, mimetypes, os

logo_path = "utils/cla_logo_white.png"
logo_html = ""
try:
    p = logo_path
    if p.startswith("dbfs:/"):
        p = "/dbfs" + p[5:]
    with open(p, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("ascii")
    mime = mimetypes.guess_type(p)[0] or "image/png"
    logo_uri = f"data:{mime};base64,{b64}"
    logo_html = f"<img src='{logo_uri}' alt='Logo' style='height:48px; width:auto; display:block;'/>"
except Exception:
    pass

displayHTML(f"""
<div style="
  background:{RED} !important;
  padding: 28px 48px;  /* Increased padding for more height and width */
  border-radius: 16px; /* Slightly larger rounded corners */
  color: {WHITE};
  text-align:center;
  margin: 12px 0 20px 0;
  max-width: 100%; /* Full width */
  display: flex;
  justify-content: center;">
  <span style="display:inline-flex; align-items:center; gap:20px;">
    {logo_html}
    <span style="font-weight:800; font-size:36px; letter-spacing:.4px; font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;">
      {title}
    </span>
  </span>
</div>
""")

In [0]:
%pip install databricks-labs-dqx==0.8.0
%pip install dbldatagen

In [0]:
dbutils.library.restartPython()

In [0]:
# =====================================================================================
# CELL 1 — demo_spec.py  (Catalog/schema, table names, schemas, MEGASPEC)
# =====================================================================================
from pyspark.sql.types import (
    StructType, StructField,
    StringType, BooleanType, DateType, TimestampType, DecimalType
)

# ──────────────────────────────────────────────────────────────────────────────
# Globals / conventions (Unity Catalog: CATALOG.SCHEMA.TABLE)
# ──────────────────────────────────────────────────────────────────────────────
DQX_CATALOG        = "dq_dev"
DQX_SCHEMA         = "demo"
DQX_DB             = f"{DQX_CATALOG}.{DQX_SCHEMA}"           # catalog.schema
QUARANTINE_TABLE   = f"{DQX_DB}.demo_quarantine"             # sink for ERROR rows
# DQX adds _warning and _error by default; we'll stick with those.
DQ_RESULT_WARN_COL = "_warning"
DQ_RESULT_ERR_COL  = "_error"

ROW_TARGETS = {
    f"{DQX_DB}.demo_employee":   2_000,
    f"{DQX_DB}.demo_customer":   1_000,
    f"{DQX_DB}.demo_project":      600,
    f"{DQX_DB}.demo_timesheet": 350_000,
    f"{DQX_DB}.demo_expense":   120_000,
}

# Canonical table names (full paths) + groupings
TABLES = {
    "catalog":    DQX_CATALOG,
    "schema":     DQX_DB,   # fully-qualified (catalog.schema)
    "employee":   f"{DQX_DB}.demo_employee",
    "customer":   f"{DQX_DB}.demo_customer",
    "project":    f"{DQX_DB}.demo_project",
    "timesheet":  f"{DQX_DB}.demo_timesheet",
    "expense":    f"{DQX_DB}.demo_expense",
    "quarantine": QUARANTINE_TABLE,
    # Groupings
    "sources": [
        f"{DQX_DB}.demo_employee",
        f"{DQX_DB}.demo_customer",
        f"{DQX_DB}.demo_project",
        f"{DQX_DB}.demo_timesheet",
        f"{DQX_DB}.demo_expense",
    ],
    "facts": [
        f"{DQX_DB}.demo_timesheet",
        f"{DQX_DB}.demo_expense",
    ],
    "dims": [
        f"{DQX_DB}.demo_employee",
        f"{DQX_DB}.demo_customer",
        f"{DQX_DB}.demo_project",
    ],
    "all": [
        f"{DQX_DB}.demo_employee",
        f"{DQX_DB}.demo_customer",
        f"{DQX_DB}.demo_project",
        f"{DQX_DB}.demo_timesheet",
        f"{DQX_DB}.demo_expense",
        QUARANTINE_TABLE,
    ],
}

# ──────────────────────────────────────────────────────────────────────────────
# Spark Structured Schemas
# ──────────────────────────────────────────────────────────────────────────────

demo_employee_schema = StructType([
    StructField("employee_id",        StringType(),  False),
    StructField("full_name",          StringType(),  False),
    StructField("department",         StringType(),  False),  # Consulting, Audit, Tax, IT, Ops
    StructField("role",               StringType(),  False),  # Engineer, Analyst, Consultant, Manager, Support
    StructField("cost_center",        StringType(),  True),   # CC-####
    StructField("employment_status",  StringType(),  False),  # Active, Leave, Terminated
    StructField("hire_date",          DateType(),    False),
    StructField("termination_date",   DateType(),    True),
    StructField("work_email",         StringType(),  True),
    StructField("country_code",       StringType(),  True),   # US, CA, MX, GB, IN
])

demo_customer_schema = StructType([
    StructField("customer_id",            StringType(),  False),
    StructField("customer_name",          StringType(),  False),
    StructField("industry",               StringType(),  False),
    StructField("country_code",           StringType(),  True),   # ISO-3166 alpha-2
    StructField("status",                 StringType(),  False),  # Active, Prospect, Inactive
    StructField("onboarding_date",        DateType(),    False),
    StructField("primary_contact_email",  StringType(),  True),
    StructField("registration_number",    StringType(),  True),   # national reg / tax-like id
])

demo_project_schema = StructType([
    StructField("project_id",          StringType(),       False),
    StructField("customer_id",         StringType(),       False),  # FK -> customer
    StructField("project_name",        StringType(),       False),
    StructField("status",              StringType(),       False),  # Planned, Active, OnHold, Closed
    StructField("start_date",          DateType(),         False),
    StructField("end_date",            DateType(),         True),
    StructField("manager_employee_id", StringType(),       True),   # FK -> employee
    StructField("budget_amount",       DecimalType(18, 2), True),
    StructField("billing_model",       StringType(),       False),  # T&M, Fixed, Retainer
])

demo_timesheet_schema = StructType([
    StructField("timesheet_id",  StringType(),       False),
    StructField("employee_id",   StringType(),       False),  # FK -> employee
    StructField("project_id",    StringType(),       False),  # FK -> project
    StructField("work_date",     DateType(),         False),
    StructField("hours_worked",  DecimalType(5, 2),  False),  # 0–24
    StructField("work_type",     StringType(),       False),  # Billable, NonBillable, Admin
    StructField("source_system", StringType(),       False),  # Workday, Jira, CSV
    StructField("created_ts",    TimestampType(),    False),
])

demo_expense_schema = StructType([
    StructField("expense_id",       StringType(),       False),
    StructField("employee_id",      StringType(),       False),  # FK -> employee
    StructField("project_id",       StringType(),       True),   # FK -> project (nullable)
    StructField("expense_date",     DateType(),         False),
    StructField("category",         StringType(),       False),  # Meals, Travel, Supplies, Software, Other
    StructField("amount",           DecimalType(18, 2), False),
    StructField("currency_code",    StringType(),       False),  # ISO-4217
    StructField("merchant",         StringType(),       True),
    StructField("receipt_attached", BooleanType(),      False),
    StructField("submission_ts",    TimestampType(),    False),
])

SCHEMAS_BY_TABLE = {
    TABLES["employee"]:  demo_employee_schema,
    TABLES["customer"]:  demo_customer_schema,
    TABLES["project"]:   demo_project_schema,
    TABLES["timesheet"]: demo_timesheet_schema,
    TABLES["expense"]:   demo_expense_schema,
}

# ──────────────────────────────────────────────────────────────────────────────
# MEGASPEC (knobs & flags used by our checks)
# ──────────────────────────────────────────────────────────────────────────────

MEGASPEC = {
    "schema": DQX_DB,
    "quarantine_table": QUARANTINE_TABLE,
    "flags": {
        "allow_weekend_billable": "warn",            # 'warn' | 'error' | 'off'
    },
    "knobs": {
        "receipt_threshold": 75.00,                  # >= threshold requires receipt
        "meal_limit": 150.00,                        # warn if Meals > limit
        "travel_limit": 500.00,                      # warn if Travel > limit
        "max_hours_per_day_error": 24.0,             # error if > this
        "hi_hours_warn": 12.0,                       # warn if > this
        "valid_country_codes": ["US","CA","MX","GB","IN"],
        "valid_currency_codes": ["USD","CAD","MXN","GBP","INR"],
        "valid_emp_status": ["Active","Leave","Terminated"],
        "valid_proj_status": ["Planned","Active","OnHold","Closed"],
        "valid_billing_models": ["T&M","Fixed","Retainer"],
        "timeliness_hours_expense_max": 240,         # submission_ts within 10 days
        "timeliness_hours_timesheet_max": 72,        # created_ts within 3 days
        "unusual_multiplier": 1.5                    # 50% higher than avg → warn
    }
}

print(f"\n[SPEC LOADED] schema={DQX_DB}  sources={len(TABLES['sources'])}  quarantine={TABLES['quarantine']}")
print("Row targets: " + ", ".join([f"{k.split('.')[-1]}={v:,}" for k,v in ROW_TARGETS.items()]))

In [0]:
# Drop all DQX tables with full catalog.schema.table names
tables_to_drop = [
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_employee",
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_customer",
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_project",
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_timesheet",
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_expense",
    f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_quarantine"
]

for t in tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS {t}")
        print(f"✅ Dropped (if existed): {t}")
    except Exception as e:
        print(f"⚠️ Skipped: {t} -> {e}")

In [0]:
# =====================================================================================
# CELL 2 — Setup (UC catalog/schema, quarantine table, banner)
# =====================================================================================

from pyspark.sql import functions as F, Window
from uuid import uuid4
from datetime import date, datetime, timedelta
from decimal import Decimal

def banner(msg, color="neon_blue"):
    col = getattr(C, color, "")
    print("\n" + "═"*92)
    print(f"{C.b}{col} {msg}{C.r}")
    print("═"*92)

banner("SETUP: Use catalog/schema and create quarantine table", "aqua_blue")

spark.sql(f"USE CATALOG {DQX_CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {DQX_DB}")
spark.sql(f"USE {DQX_DB}")

run_id = str(uuid4())
print(f"{C.ivory}Using: catalog={DQX_CATALOG}  schema={DQX_SCHEMA}  (db={DQX_DB}){C.r}")
print(f"{C.ivory}Run ID: {C.bubblegum_pink}{run_id}{C.r}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {QUARANTINE_TABLE} (
  _source_table     STRING,
  _rule_id          STRING,
  _level            STRING,
  _reason           STRING,
  _run_id           STRING,
  _event_ts         TIMESTAMP,
  _row_payload_json STRING
) USING delta
""")
print(f"{C.ivory}Quarantine table ready: {C.golden_yellow}{QUARANTINE_TABLE}{C.r}")

In [0]:
# =====================================================================================
# CELLS 3–7 — DataGen for all tables (deterministic demo rows included; avoid random "bad" noise)
# =====================================================================================
from dbldatagen import DataGenerator
from utils.display import show_df  # << use your helper for nice display
from datetime import date, datetime, timedelta
from decimal import Decimal
from uuid import uuid4

# -------------------------------------------------------------------------------------
# CELL 3 — Employees
# -------------------------------------------------------------------------------------
banner(f"BUILD (Datagen): {TABLES['employee']}", "sky_blue")
n_emp = ROW_TARGETS[TABLES["employee"]]

emp_gen = (DataGenerator(spark, name="demo_employee_gen", rows=n_emp, partitions=8)
           .withIdOutput()
           .withColumn("employee_id", "string", expr="concat('E', lpad(cast(id + 1001 as string), 4, '0'))")
           .withColumn("full_name", "string", expr="concat('Emp ', cast(id as string))")
           .withColumn("department", values=["Consulting","Audit","Tax","IT","Ops"], weights=[0.35,0.20,0.20,0.15,0.10])
           .withColumn("role", values=["Engineer","Analyst","Consultant","Manager","Support"])
           .withColumn("cost_center", "string", expr="concat('CC-', lpad(cast(cast(rand()*10000 as int) as string), 4, '0'))", percentNulls=0.02)
           .withColumn("employment_status", values=MEGASPEC["knobs"]["valid_emp_status"], weights=[0.90,0.03,0.07])
           .withColumn("hire_date", "date", minValue="2018-01-01", maxValue="2025-08-01")
           .withColumn("termination_date", "date", minValue="2018-02-01", maxValue="2030-12-31", percentNulls=1.0)
           .withColumn("work_email", "string", expr="concat('emp', cast(id as string), '@company.com')")
           .withColumn("country_code", values=MEGASPEC["knobs"]["valid_country_codes"])
)
emp_df = emp_gen.build().drop("id")

# Deterministic demo employees (specific rule hits)
demo_rows_emp = [
    ("E9999", "Terminated_NoEnd", "Consulting", "Analyst", "CC-0001",
     "Terminated", date(2023,1,15), None, "emp9999@company.com", "US"),
    ("E9998", "Terminated_BadEnd", "Audit", "Engineer", "CC-0002",
     "Terminated", date(2023,6,1), date(2023,5,15), "emp9998@company.com", "US"),
    ("E9997", "ExtEmail_InvalidCC", "IT", "Manager", "CC-0003",
     "Active", date(2021,3,1), None, "someone@gmail.com", "XX"),
]
emp_demo_df = spark.createDataFrame(demo_rows_emp, schema=SCHEMAS_BY_TABLE[TABLES["employee"]])
emp_df = emp_df.unionByName(emp_demo_df)

emp_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["employee"])
print(f"{C.ivory}Wrote: {C.golden_yellow}{TABLES['employee']}{C.r}  rows={spark.table(TABLES['employee']).count():,}")
show_df(spark.table(TABLES["employee"]), n=20, truncate=False)

# -------------------------------------------------------------------------------------
# CELL 4 — Customers
# -------------------------------------------------------------------------------------
banner(f"BUILD (Datagen): {TABLES['customer']}", "sky_blue")
n_cust = ROW_TARGETS[TABLES["customer"]]

cust_gen = (DataGenerator(spark, name="demo_customer_gen", rows=n_cust, partitions=8)
            .withIdOutput()
            .withColumn("customer_id", "string", expr="concat('C', lpad(cast(id + 5001 as string), 4, '0'))")
            .withColumn("customer_name", "string", expr="concat('Customer ', cast(id as string))")
            .withColumn("industry", values=["Technology","Healthcare","Finance","Retail","Manufacturing"], weights=[0.30,0.20,0.20,0.20,0.10])
            .withColumn("country_code", values=MEGASPEC["knobs"]["valid_country_codes"])
            .withColumn("status", values=["Active","Prospect","Inactive"], weights=[0.75,0.15,0.10])
            .withColumn("onboarding_date", "date", minValue="2019-01-01", maxValue="2025-08-01")
            .withColumn("primary_contact_email", "string", expr="concat('contact', cast(id as string), '@example.com')")
            .withColumn("registration_number", "string", expr="concat('RN-', lpad(cast(cast(rand()*100000000 as int) as string), 8, '0'))")
)
cust_df = cust_gen.build().drop("id")

# Deterministic dup registration among Active
demo_rows_cust = [
    ("C9999", "DupReg A", "Technology", "US", "Active",  date(2024,1,1), "a@example.com", "RN-00001234"),
    ("C9998", "DupReg B", "Technology", "US", "Active",  date(2024,1,2), "b@example.com", "RN-00001234"),
]
cust_demo_df = spark.createDataFrame(demo_rows_cust, schema=SCHEMAS_BY_TABLE[TABLES["customer"]])
cust_df = cust_df.unionByName(cust_demo_df)

cust_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["customer"])
print(f"{C.ivory}Wrote: {C.golden_yellow}{TABLES['customer']}{C.r}  rows={spark.table(TABLES['customer']).count():,}")
show_df(spark.table(TABLES["customer"]), n=20, truncate=False)

# -------------------------------------------------------------------------------------
# CELL 5 — Projects
# -------------------------------------------------------------------------------------
banner(f"BUILD (Datagen): {TABLES['project']}", "sky_blue")
n_proj = ROW_TARGETS[TABLES["project"]]

emp_ids  = [r[0] for r in spark.table(TABLES["employee"]).select("employee_id").collect()]
cust_ids = [r[0] for r in spark.table(TABLES["customer"]).select("customer_id").collect()]
mgr_id   = emp_ids[0] if emp_ids else None

proj_gen = (DataGenerator(spark, name="demo_project_gen", rows=n_proj, partitions=8)
            .withIdOutput()
            .withColumn("project_id", "string", expr="concat('P', lpad(cast(id + 10001 as string), 5, '0'))")
            .withColumn("customer_id", values=cust_ids)
            .withColumn("project_name", "string", expr="concat('Project ', cast(id as string))")
            .withColumn("status", values=MEGASPEC["knobs"]["valid_proj_status"], weights=[0.15,0.55,0.10,0.20])
            .withColumn("start_date", "date", minValue="2020-01-01", maxValue="2025-03-01")
            .withColumn("end_date", "date", minValue="2020-02-01", maxValue="2028-12-31", percentNulls=0.35)
            .withColumn("manager_employee_id", values=emp_ids, percentNulls=0.03)
            .withColumn("budget_amount", "decimal(18,2)", minValue=10000, maxValue=2000000)
            .withColumn("billing_model", values=MEGASPEC["knobs"]["valid_billing_models"], weights=[0.55,0.35,0.10])
)
proj_df = proj_gen.build().drop("id")

# Deterministic demo projects (specific rule hits)
demo_rows_proj = [
    ("P99998", cust_ids[0] if cust_ids else "C9999", "Demo_EndBeforeStart", "Active",
     date(2024,5,1), date(2024,4,15), mgr_id, Decimal("250000.00"), "T&M"),
    ("P99999", "C0000", "Demo_MissingCustomer", "Active",
     date(2024,6,1), None, mgr_id, Decimal("150000.00"), "Fixed"),
    ("P99997", cust_ids[0] if cust_ids else "C9999", "Demo_ClosedNoEnd", "Closed",
     date(2024,2,1), None, mgr_id, Decimal("100000.00"), "Retainer"),
]
proj_demo_df = spark.createDataFrame(demo_rows_proj, schema=SCHEMAS_BY_TABLE[TABLES["project"]])
proj_df = proj_df.unionByName(proj_demo_df)

proj_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["project"])
print(f"{C.ivory}Wrote: {C.golden_yellow}{TABLES['project']}{C.r}  rows={spark.table(TABLES['project']).count():,}")
show_df(spark.table(TABLES["project"]), n=20, truncate=False)

# -------------------------------------------------------------------------------------
# CELL 6 — Timesheets  (enforce DECIMAL(5,2) + overwriteSchema)
# -------------------------------------------------------------------------------------
banner(f"BUILD (Datagen): {TABLES['timesheet']}", "sky_blue")
n_ts = ROW_TARGETS[TABLES["timesheet"]]

emp_ids = [r[0] for r in spark.table(TABLES["employee"]).select("employee_id").collect()]
proj_keys = (spark.table(TABLES["project"]).select("project_id","start_date","end_date").collect())
project_id_list = [r["project_id"] for r in proj_keys]
proj_map = {r["project_id"]: (r["start_date"], r["end_date"]) for r in proj_keys}

ts_gen = (DataGenerator(spark, name="demo_timesheet_gen", rows=n_ts, partitions=48)
          .withColumn("timesheet_id", "string", expr="uuid()")
          .withColumn("employee_id", values=emp_ids)
          .withColumn("project_id", values=project_id_list)
          .withColumn("work_date", "date", minValue="2024-01-01", maxValue="2025-08-10")
          .withColumn("hours_worked", "decimal(5,2)", minValue=0.00, maxValue=12.00)
          .withColumn("work_type", values=["Billable","NonBillable","Admin"], weights=[0.70,0.25,0.05])
          .withColumn("source_system", values=["Workday","Jira","CSV"], weights=[0.70,0.20,0.10])
          .withColumn("created_ts", "timestamp")
)
ts_df = ts_gen.build()

# created_ts near work_date (0–72 hours)
ts_df = ts_df.withColumn(
    "created_ts",
    F.expr("timestamp(work_date) + make_interval(0,0,0,cast(rand()*3 as int), cast(rand()*24 as int), cast(rand()*60 as int), 0)")
)

# Deterministic demo timesheets
chosen_emp = emp_ids[0] if emp_ids else None
chosen_proj = project_id_list[0] if project_id_list else None
if chosen_emp and chosen_proj:
    p_start, _ = proj_map.get(chosen_proj, (date(2024,1,1), None))
    demo_rows_ts = [
        (str(uuid4()), chosen_emp, chosen_proj, p_start - timedelta(days=3),
         Decimal("8.00"), "Billable", "Workday", datetime.utcnow()),
        (str(uuid4()), chosen_emp, chosen_proj, date.today() - timedelta(days=10),
         Decimal("8.00"), "Billable", "Workday", datetime.utcnow()),
        (str(uuid4()), chosen_emp, chosen_proj, date.today() - timedelta(days=10),
         Decimal("8.00"), "Billable", "Workday", datetime.utcnow()),
    ]
    ts_demo_df = spark.createDataFrame(demo_rows_ts, schema=SCHEMAS_BY_TABLE[TABLES["timesheet"]])
    ts_df = ts_df.unionByName(ts_demo_df)

# Enforce canonical schema & write with overwriteSchema
ts_schema = SCHEMAS_BY_TABLE[TABLES["timesheet"]]
for field in ts_schema:
    ts_df = ts_df.withColumn(field.name, F.col(field.name).cast(field.dataType))

ts_df.select([f.name for f in ts_schema]) \
     .write.format("delta") \
     .mode("overwrite") \
     .option("overwriteSchema", "true") \
     .saveAsTable(TABLES["timesheet"])

print(f"{C.ivory}Wrote: {C.golden_yellow}{TABLES['timesheet']}{C.r}  rows={spark.table(TABLES['timesheet']).count():,}")
show_df(spark.table(TABLES["timesheet"]), n=20, truncate=False)

# -------------------------------------------------------------------------------------
# CELL 7 — Expenses
# -------------------------------------------------------------------------------------
banner(f"BUILD (Datagen): {TABLES['expense']}", "sky_blue")
n_exp = ROW_TARGETS[TABLES["expense"]]
emp_ids = [r[0] for r in spark.table(TABLES["employee"]).select("employee_id").collect()]
proj_ids = [r[0] for r in spark.table(TABLES["project"]).select("project_id").collect()]

exp_gen = (DataGenerator(spark, name="demo_expense_gen", rows=n_exp, partitions=24)
           .withColumn("expense_id", "string", expr="uuid()")
           .withColumn("employee_id", values=emp_ids)
           .withColumn("project_id", values=proj_ids, percentNulls=0.25)
           .withColumn("expense_date", "date", minValue="2024-01-01", maxValue="2025-08-10")
           .withColumn("category", values=["Meals","Travel","Supplies","Software","Other"], weights=[0.35,0.25,0.20,0.10,0.10])
           .withColumn("amount", "decimal(18,2)", minValue=5.00, maxValue=5000.00)
           .withColumn("currency_code", values=MEGASPEC["knobs"]["valid_currency_codes"])
           .withColumn("merchant", values=["Uber","Lyft","Delta","AA","Staples","BestBuy","Amazon","LocalCafe","HotelCo","SoftwareCo"])
           .withColumn("receipt_attached", "boolean", expr="rand() > 0.08")
           .withColumn("submission_ts", "timestamp")
)
exp_df = exp_gen.build()

# submission_ts near expense_date (0–240 hours)
exp_df = exp_df.withColumn(
    "submission_ts",
    F.expr("timestamp(expense_date) + make_interval(0,0,0,cast(rand()*10 as int), cast(rand()*24 as int), cast(rand()*60 as int), 0)")
)

# Deterministic demo expenses
if emp_ids:
    demo_emp = emp_ids[0]
    emp_cc = (spark.table(TABLES["employee"]).where(F.col("employee_id")==demo_emp)
              .select("country_code").first())
    home_cc = emp_cc["country_code"] if emp_cc and emp_cc["country_code"] else "US"
    home_ccy = {"US":"USD","CA":"CAD","MX":"MXN","GB":"GBP","IN":"INR"}.get(home_cc, "USD")
    away_ccy = "GBP" if home_ccy != "GBP" else "USD"
else:
    demo_emp = "E9999"; home_ccy = "USD"; away_ccy = "GBP"

demo_proj = proj_ids[0] if proj_ids else None
today = date.today()
demo_rows_exp = [
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=20), "Meals", Decimal("15.00"), home_ccy, "LocalCafe", False, datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=18), "Meals", Decimal("22.00"), home_ccy, "LocalCafe", True,  datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=16), "Meals", Decimal("18.00"), home_ccy, "LocalCafe", True,  datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=14), "Meals", Decimal("20.50"), home_ccy, "LocalCafe", True,  datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=1),  "Meals", Decimal("600.00"), away_ccy, "LocalCafe", True,  datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=2),  "Meals", Decimal("24.99"), home_ccy, "LocalCafe", False, datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=2),  "Meals", Decimal("24.99"), home_ccy, "LocalCafe", False, datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=2),  "Meals", Decimal("26.99"), home_ccy, "LocalCafe", False, datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=5),  "Supplies", Decimal("129.00"), home_ccy, "Amazon", True,  datetime.utcnow()),
    (str(uuid4()), demo_emp, demo_proj, today - timedelta(days=5),  "Supplies", Decimal("129.00"), home_ccy, "Amazon", True,  datetime.utcnow()),
]
exp_demo_df = spark.createDataFrame(demo_rows_exp, schema=SCHEMAS_BY_TABLE[TABLES["expense"]])
exp_df = exp_df.unionByName(exp_demo_df)

exp_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["expense"])
print(f"{C.ivory}Wrote: {C.golden_yellow}{TABLES['expense']}{C.r}  rows={spark.table(TABLES['expense']).count():,}")
show_df(spark.table(TABLES["expense"]), n=20, truncate=False)

---

## Testing

In [0]:
# === Cell 1: Install DQX Package (if your cluster image doesn't already include it) ===
%pip install databricks-labs-dqx==0.8.0

In [0]:
# === Cell 2: Restart Python to pick up libs (Databricks convention) ===
dbutils.library.restartPython()

### Load DQX Config

In [0]:
import os
import json
import hashlib
import yaml
from typing import Dict, Any, Optional

from pyspark.sql import SparkSession, types as T
from delta.tables import DeltaTable
from databricks.labs.dqx.engine import DQEngine
from pyspark.sql.functions import to_timestamp, col

from utils.print import print_notebook_env
from utils.timezone import current_time_iso


# --- Unified schema, with DQX columns sandwiched ---
# DQX core columns must match the docs:
# name (STRING), criticality (STRING), check (STRUCT), filter (STRING),
# run_config_name (STRING), user_metadata (MAP<STRING,STRING>)
TABLE_SCHEMA = T.StructType([
    T.StructField("hash_id", T.StringType(), False),
    T.StructField("table_name", T.StringType(), False),

    # DQX fields begin here
    T.StructField("name", T.StringType(), False),
    T.StructField("criticality", T.StringType(), False),
    T.StructField(
        "check",
        T.StructType([
            T.StructField("function", T.StringType(), False),
            T.StructField("for_each_column", T.ArrayType(T.StringType()), True),
            T.StructField("arguments", T.MapType(T.StringType(), T.StringType()), True),
        ]),
        False,
    ),
    T.StructField("filter", T.StringType(), True),
    T.StructField("run_config_name", T.StringType(), False),
    T.StructField("user_metadata", T.MapType(T.StringType(), T.StringType()), True),

    # your ops fields
    T.StructField("yaml_path", T.StringType(), False),
    T.StructField("active", T.BooleanType(), False),
    T.StructField("created_by", T.StringType(), False),
    T.StructField("created_at", T.StringType(), False),  # stored as ISO string; we may cast on write
    T.StructField("updated_by", T.StringType(), True),
    T.StructField("updated_at", T.StringType(), True),
])


def compute_hash(rule_dict: Dict[str, Any]) -> str:
    """Stable hash over the identifying fields of a rule."""
    relevant = {
        k: rule_dict[k]
        for k in ["table_name", "name", "criticality", "run_config_name", "check"]
        if k in rule_dict
    }
    return hashlib.md5(json.dumps(relevant, sort_keys=True).encode()).hexdigest()


def _stringify_map_values(d: Dict[str, Any]) -> Dict[str, str]:
    """
    Convert a dict of arbitrary JSON-serializable values to map<string,string>
    required by DQX (lists/dicts -> JSON, bool -> 'true'/'false', else str()).
    """
    out: Dict[str, str] = {}
    for k, v in (d or {}).items():
        if isinstance(v, (list, dict)):
            out[k] = json.dumps(v)
        elif isinstance(v, bool):
            out[k] = "true" if v else "false"
        elif v is None:
            out[k] = "null"
        else:
            out[k] = str(v)
    return out


def process_yaml_file(path: str, output_config: Dict[str, Any], time_zone: str = "UTC"):
    """Read one YAML file, validate, and flatten into rows for the table."""
    file_base = os.path.splitext(os.path.basename(path))[0]
    with open(path, "r") as fh:
        docs = yaml.safe_load(fh)
    if isinstance(docs, dict):
        docs = [docs]

    validate_rules_file(docs, file_base, path)

    now = current_time_iso(time_zone)
    flat_rules = []

    for rule in docs:
        validate_rule_fields(rule, path)

        h = compute_hash(rule)
        check_dict = rule["check"]

        # Strong typing for DQX struct:
        function = check_dict.get("function")
        if not isinstance(function, str) or not function:
            raise ValueError(f"{path}: check.function must be a non-empty string (rule '{rule.get('name')}').")

        for_each = check_dict.get("for_each_column")
        if for_each is not None and not isinstance(for_each, list):
            raise ValueError(f"{path}: check.for_each_column must be an array of strings (rule '{rule.get('name')}').")
        if isinstance(for_each, list):
            try:
                for_each = [str(x) for x in for_each]
            except Exception:
                raise ValueError(f"{path}: unable to cast for_each_column items to strings (rule '{rule.get('name')}').")

        arguments = check_dict.get("arguments", {}) or {}
        if not isinstance(arguments, dict):
            raise ValueError(f"{path}: check.arguments must be a map (rule '{rule.get('name')}').")
        arguments = _stringify_map_values(arguments)  # enforce map<string,string>

        user_metadata = rule.get("user_metadata")
        if user_metadata is not None:
            if not isinstance(user_metadata, dict):
                raise ValueError(f"{path}: user_metadata must be a map<string,string> (rule '{rule.get('name')}').")
            user_metadata = _stringify_map_values(user_metadata)

        flat_rules.append({
            "hash_id": h,
            "table_name": rule["table_name"],

            "name": rule["name"],
            "criticality": rule["criticality"],
            "check": {
                "function": function,
                "for_each_column": for_each if for_each else None,
                "arguments": arguments if arguments else None,
            },
            "filter": rule.get("filter"),
            "run_config_name": rule["run_config_name"],
            "user_metadata": user_metadata if user_metadata else None,

            "yaml_path": path,
            "active": rule.get("active", True),
            "created_by": "AdminUser",
            "created_at": now,
            "updated_by": None,
            "updated_at": None,
        })

    # Validate with DQX engine (semantic)
    validate_with_dqx(docs, path)
    return flat_rules


def parse_output_config(config_path: str) -> Dict[str, Any]:
    with open(config_path, "r") as fh:
        config = yaml.safe_load(fh)
    # Expect the new keys
    required = ["dqx_checks_config_table_name", "dqx_yaml_checks", "run_config_name"]
    missing = [k for k in required if k not in config]
    if missing:
        raise ValueError(f"Config missing required keys: {missing}")
    return config


def validate_rules_file(rules, file_base: str, file_path: str):
    problems = []
    seen_names = set()
    table_names = {r.get("table_name") for r in rules if isinstance(r, dict)}
    if len(table_names) != 1:
        problems.append(f"Inconsistent table_name values in {file_path}: {table_names}")
    expected_table = file_base
    try:
        tn = list(table_names)[0]
        if tn.split(".")[-1] != expected_table:
            problems.append(
                f"Table name in rules ({tn}) does not match filename ({expected_table}) in {file_path}"
            )
    except Exception:
        problems.append(f"No valid table_name found in {file_path}")

    for rule in rules:
        name = rule.get("name")
        if not name:
            problems.append(f"Missing rule name in {file_path}")
        if name in seen_names:
            problems.append(f"Duplicate rule name '{name}' in {file_path}")
        seen_names.add(name)

    if problems:
        raise ValueError(f"File-level validation failed in {file_path}: {problems}")


def validate_rule_fields(rule, file_path: str):
    problems = []
    required_fields = ["table_name", "name", "criticality", "run_config_name", "check"]
    for field in required_fields:
        if not rule.get(field):
            problems.append(
                f"Missing required field '{field}' in rule '{rule.get('name')}' ({file_path})"
            )
    if rule.get("table_name", "").count(".") != 2:
        problems.append(
            f"table_name '{rule.get('table_name')}' not fully qualified in rule '{rule.get('name')}' ({file_path})"
        )
    if rule.get("criticality") not in {"error", "warn", "warning"}:
        problems.append(
            f"Invalid criticality '{rule.get('criticality')}' in rule '{rule.get('name')}' ({file_path})"
        )
    if not rule.get("check", {}).get("function"):
        problems.append(
            f"Missing check.function in rule '{rule.get('name')}' ({file_path})"
        )
    if problems:
        raise ValueError(f"Rule-level validation failed: {problems}")


def validate_with_dqx(rules, file_path: str):
    status = DQEngine.validate_checks(rules)
    if status.has_errors:
        raise ValueError(f"DQX validation failed in {file_path}:\n{status.to_string()}")


def ensure_delta_table(spark: SparkSession, delta_table_name: str):
    if not spark.catalog.tableExists(delta_table_name):
        print(f"Creating new Delta table at {delta_table_name}")
        empty_df = spark.createDataFrame([], TABLE_SCHEMA)
        empty_df.write.format("delta").saveAsTable(delta_table_name)
    else:
        print(f"Delta table already exists at {delta_table_name}")


def upsert_rules_into_delta(spark: SparkSession, rules, delta_table_name: str):
    if not rules:
        print("No rules to write, skipping upsert.")
        return

    print(f"\nWriting rules to Delta table '{delta_table_name}'...")
    print(f"Number of rules to write: {len(rules)}")

    df = spark.createDataFrame(rules, schema=TABLE_SCHEMA)

    # Cast audit timestamps to actual TIMESTAMP in the sink
    df = df.withColumn("created_at", to_timestamp(col("created_at"))) \
           .withColumn("updated_at", to_timestamp(col("updated_at")))

    try:
        delta_table = DeltaTable.forName(spark, delta_table_name)
        delta_table.alias("target").merge(
            df.alias("source"),
            "target.yaml_path = source.yaml_path AND target.table_name = source.table_name AND target.name = source.name"
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    except Exception:
        print("Delta merge failed (likely first write). Writing full table.")
        df.write.format("delta").saveAsTable(delta_table_name)

    print(f"Successfully wrote {df.count()} rules to '{delta_table_name}'.")


def print_rules_df(spark: SparkSession, rules):
    if not rules:
        print("No rules to show.")
        return
    df = spark.createDataFrame(rules, schema=TABLE_SCHEMA)
    df = df.withColumn("created_at", to_timestamp(col("created_at"))) \
           .withColumn("updated_at", to_timestamp(col("updated_at")))
    print("\n==== Dry Run: Rules DataFrame to be uploaded ====")
    df.show(truncate=False, n=50)
    print(f"Total rules: {df.count()}")
    return df


def validate_all_rules(rules_dir: str, output_config: Dict[str, Any], fail_fast: bool = True):
    errors = []
    print(f"Starting validation for all YAML rule files in '{rules_dir}'")
    for fname in os.listdir(rules_dir):
        if not fname.endswith((".yaml", ".yml")):
            continue
        full_path = os.path.join(rules_dir, fname)
        print(f"\nValidating file: {full_path}")
        try:
            file_base = os.path.splitext(os.path.basename(full_path))[0]
            with open(full_path, "r") as fh:
                docs = yaml.safe_load(fh)
            if isinstance(docs, dict):
                docs = [docs]
            validate_rules_file(docs, file_base, full_path)
            print(f"  File-level validation passed for {full_path}")
            for rule in docs:
                validate_rule_fields(rule, full_path)
                print(f"    Rule-level validation passed for rule '{rule.get('name')}'")
            validate_with_dqx(docs, full_path)
            print(f"  DQX validation PASSED for {full_path}")
        except Exception as ex:
            print(f"  Validation FAILED for file {full_path}\n  Reason: {ex}")
            errors.append(str(ex))
            if fail_fast:
                break
    if not errors:
        print("\nAll YAML rule files are valid!")
    else:
        print("\nRule validation errors found:")
        for e in errors:
            print(e)
    return errors


def main(
    output_config_path: str = "resources/dqx_config.yaml",
    rules_dir: Optional[str] = None,
    time_zone: str = "America/Chicago",
    dry_run: bool = False,
    validate_only: bool = False
):
    spark = SparkSession.builder.getOrCreate()

    print_notebook_env(spark, local_timezone=time_zone)

    output_config = parse_output_config(output_config_path)

    # pick up rules_dir from config if not provided explicitly
    rules_dir = rules_dir or output_config["dqx_yaml_checks"]

    delta_table_name = output_config["dqx_checks_config_table_name"]

    all_rules = []
    for fname in os.listdir(rules_dir):
        if not fname.endswith((".yaml", ".yml")):
            continue
        full_path = os.path.join(rules_dir, fname)
        file_rules = process_yaml_file(full_path, output_config, time_zone=time_zone)
        all_rules.extend(file_rules)

    if validate_only:
        print("\nValidation only: not writing any rules.")
        validate_all_rules(rules_dir, output_config)
        return

    if dry_run:
        print_rules_df(spark, all_rules)
        return

    ensure_delta_table(spark, delta_table_name)
    upsert_rules_into_delta(spark, all_rules, delta_table_name)
    print(f"Finished writing rules to '{delta_table_name}'.")


if __name__ == "__main__":
    # main(dry_run=True)
    # main(validate_only=True)
    main()

### 02_run_dqx_checks

In [0]:
# dqx/info/two_rules_test.yaml

# Row-level: Filter rows where 'result_state'='FAILED' & Annotate where 'state'='TERMINATED'
- table_name: dq_prd.monitoring.job_run_audit
  name: failed_state_is_terminated
  criticality: error
  run_config_name: default
  filter: "result_state = 'FAILED'"
  check:
    function: sql_expression
    arguments:
      expression: "state = 'TERMINATED'"

# Dataset-level: composite uniqueness (one run per job+start_time)
- table_name: dq_prd.monitoring.job_run_audit
  name: unique_job_id_start_time
  criticality: error
  run_config_name: default
  check:
    function: is_unique
    arguments:
      columns: [job_id, start_time]

In [0]:
import yaml
from collections import defaultdict
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from pyspark.sql import functions as F

yaml_path = "two_rules_test.yaml"

# 1) Load YAML
with open(yaml_path, "r") as fh:
    rules = yaml.safe_load(fh) or []
if isinstance(rules, dict):
    rules = [rules]

# 2) Group by table
by_table = defaultdict(list)
for r in rules:
    t = r.get("table_name")
    if t:
        by_table[t].append(r)

print(f"Loaded {sum(len(v) for v in by_table.values())} rule(s) across {len(by_table)} table(s): {list(by_table.keys())}")

engine = DQEngine(WorkspaceClient())

# 3) Apply per table
for table, tbl_rules in by_table.items():
    print(f"\n=== Applying {len(tbl_rules)} rule(s) to {table} ===")
    df = spark.table(table)

    annotated = engine.apply_checks_by_metadata(df, tbl_rules)

    # DQX may use _error/_warning or _errors/_warnings depending on version
    err_col = "_errors"  if "_errors"  in annotated.columns else "_error"
    wrn_col = "_warnings" if "_warnings" in annotated.columns else "_warning"

    # Summary
    agg = (
        annotated.select(
            F.size(F.col(err_col)).alias("e_sz"),
            F.size(F.col(wrn_col)).alias("w_sz"),
        )
        .agg(
            F.coalesce(F.sum("e_sz"), F.lit(0)).alias("errors"),
            F.coalesce(F.sum("w_sz"), F.lit(0)).alias("warnings"),
        )
        .collect()[0]
    )
    print(f"Hit summary -> errors={int(agg['errors'])}, warnings={int(agg['warnings'])}")

    # Keep only rows that actually have hits
    hits_only = annotated.filter(
        (F.col(err_col).isNotNull() & (F.size(F.col(err_col)) > 0)) |
        (F.col(wrn_col).isNotNull() & (F.size(F.col(wrn_col)) > 0))
    )

    # Explode per array (no _outer), then union; drop all-null just in case
    errors = (
        hits_only.filter(F.size(F.col(err_col)) > 0)
        .select(F.explode(F.col(err_col)).alias("r"))
        .select("r.*")
    )
    warnings = (
        hits_only.filter(F.size(F.col(wrn_col)) > 0)
        .select(F.explode(F.col(wrn_col)).alias("r"))
        .select("r.*")
    )
    hits = errors.unionByName(warnings, allowMissingColumns=True).na.drop("all")

    print("\nSummary by rule:")
    hits.groupBy("name", "function").count().orderBy(F.desc("count")).show(truncate=False)

    print("\nViolations (one row per hit):")
    try:
        display(hits.select("name", "message", "columns", "filter", "function", "run_time", "user_metadata"))
    except NameError:
        hits.select("name", "message", "columns", "filter", "function", "run_time", "user_metadata").show(truncate=False)

In [0]:
import yaml
from collections import defaultdict
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.config import TableChecksStorageConfig
from pyspark.sql import functions as F

# ============
# Define checks inline (YAML string → Python list[dict])
# Update table_name if needed
# ============
checks = yaml.safe_load("""
- table_name: dq_prd.monitoring.job_run_audit
  name: failed_state_is_terminated
  criticality: error
  run_config_name: default
  filter: "result_state = 'FAILED'"
  check:
    function: sql_expression
    arguments:
      expression: "state = 'TERMINATED'"

- table_name: dq_prd.monitoring.job_run_audit
  name: failed_runs_have_error_code
  criticality: error
  run_config_name: default
  filter: "result_state = 'FAILED'"
  check:
    function: sql_expression
    arguments:
      expression: "error_code IS NOT NULL AND trim(error_code) <> ''"
""")

# ============
# Validate with DQX (will catch wrong types/args early)
# ============
status = DQEngine.validate_checks(checks)
if getattr(status, "has_errors", False):
    print("Validation errors:\n" + status.to_string())
    raise SystemExit(1)

# ============
# Group by table and run
# ============
by_table = defaultdict(list)
for r in checks:
    by_table[r["table_name"]].append(r)

engine = DQEngine(WorkspaceClient())

for table, tbl_rules in by_table.items():
    print(f"\n=== Applying {len(tbl_rules)} rule(s) to {table} ===")
    df = spark.table(table)

    annotated = engine.apply_checks_by_metadata(df, tbl_rules)

    # DQX may expose _error/_warning or _errors/_warnings depending on version
    err_col = "_errors" if "_errors" in annotated.columns else "_error"
    wrn_col = "_warnings" if "_warnings" in annotated.columns else "_warning"

    # Quick summary
    agg = (
        annotated.select(
            F.size(F.col(err_col)).cast("long").alias("e_sz"),
            F.size(F.col(wrn_col)).cast("long").alias("w_sz"),
        )
        .agg(
            F.coalesce(F.sum("e_sz"), F.lit(0).cast("long")).alias("errors"),
            F.coalesce(F.sum("w_sz"), F.lit(0).cast("long")).alias("warnings"),
        )
        .collect()[0]
    )
    print(f"Hit summary -> errors={int(agg['errors'])}, warnings={int(agg['warnings'])}")

    # Show per-rule counts
    hits_only = annotated.filter(
        (F.col(err_col).isNotNull() & (F.size(F.col(err_col)) > 0)) |
        (F.col(wrn_col).isNotNull() & (F.size(F.col(wrn_col)) > 0))
    )
    errors = (
        hits_only.filter(F.size(F.col(err_col)) > 0)
        .select(F.explode(F.col(err_col)).alias("r"))
        .select("r.*")
    )
    warnings = (
        hits_only.filter(F.size(F.col(wrn_col)) > 0)
        .select(F.explode(F.col(wrn_col)).alias("r"))
        .select("r.*")
    )
    hits = errors.unionByName(warnings, allowMissingColumns=True)

    print("\nSummary by rule:")
    hits.groupBy("name", "function").count().orderBy(F.desc("count")).show(truncate=False)

    print("\nSample violations:")
    hits.select("name", "message", "columns", "filter", "function", "run_time", "user_metadata").show(20, truncate=False)

# ============
# OPTIONAL: save these inline checks into your config table for future runs
# (uncomment to persist)
# ============
# engine.save_checks(
#     checks,
#     config=TableChecksStorageConfig(
#         location="dq_dev.dqx.checks_config",   # <-- set to your checks Delta table FQN
#         run_config_name="default",
#         mode="append"                          # or "overwrite"
#     )
# )
# print("Saved checks to dq_dev.dqx.checks_config")

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQRowRule
from databricks.labs.dqx import check_funcs

# Target table → DataFrame
df = spark.table("dq_prd.monitoring.job_run_audit")

# Define checks as DQX classes
checks = [
    DQRowRule(
        name="run_id_is_not_null",
        criticality="error",
        check_func=check_funcs.is_not_null,
        column="run_id",
    ),
    DQRowRule(
        name="failed_state_is_terminated",
        criticality="error",
        filter="result_state = 'FAILED'",
        check_func=check_funcs.sql_expression,
        check_func_kwargs={
            "expression": "state = 'TERMINATED'",  # assert condition; fails when not met
            "msg": "FAILED run not in TERMINATED state"
        },
    ),
]

# Apply checks → adds `_warning` / `_error` columns at the end
dq = DQEngine(WorkspaceClient())
df_checked = dq.apply_checks(df, checks)

# Show the result; rows with issues will have non-empty arrays in `_error` / `_warning`
display(df_checked)
# If you only want affected rows during testing:
# display(df_checked.where((F.size("_error") > 0) | (F.size("_warning") > 0)))

### Run DQX & Fingerprint Results

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.config import FileChecksStorageConfig
from pyspark.sql import functions as F

# 1) Target table → DF
df = spark.table("de_prd.gold.wkdy_dim_project")

# 2) Load checks from YAML (adjust path if your cwd differs)
dq = DQEngine(WorkspaceClient())
checks = dq.load_checks(
    FileChecksStorageConfig(location="wkdy_dim_project.yaml")
)

# 3) Apply checks → appends `_warning` / `_error` array<struct> columns
df_checked = dq.apply_checks_by_metadata(df, checks)

# Show only affected rows (keep full table shape, just filtered)
#display(df_checked.where((F.size("_error") > 0) | (F.size("_warning") > 0)))

# If you want everything, not filtered:
display(df_checked)

# (Optional) Need a flat “result dataframe” of issues matching your schema?
# errs = (df_checked
#   .select(F.explode(F.array_union("_error","_warning")).alias("r"))
#   .select("r.name","r.message","r.columns","r.filter","r.function","r.run_time","r.user_metadata"))
# display(errs)

In [0]:
# keep rows where _errors exists and has items
df_filtered = df_checked.where(F.col("_errors").isNotNull() & (F.size("_errors") > 0))
display(df_filtered)

In [0]:
df_COUNT = df_filtered.count()
print(f"There are {df_COUNT:,} rows with errors")

In [0]:
from pyspark.sql import functions as F

# Pick the actual column names DQX created
err_col  = "_errors"  if "_errors"  in df_checked.columns else "_error"
warn_col = "_warnings" if "_warnings" in df_checked.columns else "_warning"

# Rows with at least one error
df_filtered = df_checked.where(F.size(F.col(err_col)) > 0)

# 1) Total rows that have any error
rows_with_errors = df_filtered.count()
print(f"Rows with any error: {rows_with_errors:,}")

# 2) Count violations per error rule
error_counts = (
    df_filtered
      .select(F.explode(F.col(err_col)).alias("e"))
      .groupBy(F.col("e.name").alias("rule"))
      .count()
      .withColumnRenamed("count", "violations")
      .orderBy(F.desc("violations"))
)
display(error_counts)

# 3) (Optional) warnings breakdown too
if warn_col in df_checked.columns:
    warning_counts = (
        df_checked.where(F.size(F.col(warn_col)) > 0)
          .select(F.explode(F.col(warn_col)).alias("w"))
          .groupBy(F.col("w.name").alias("rule"))
          .count()
          .withColumnRenamed("count", "violations")
          .orderBy(F.desc("violations"))
    )
    display(warning_counts)

# 4) (Optional) total violation events (sum across all rules; a single row can contribute >1)
total_error_events = (
    df_filtered
      .select(F.explode(F.col(err_col)).alias("e"))
      .count()
)
print(f"Total error events (all rules): {total_error_events:,}")

In [0]:
##############################################
-- check_name: project_type_is_not_null_or_unknown
SELECT COUNT(*) AS violation_count
FROM de_prd.gold.wkdy_dim_project
WHERE project_type_name IS NULL
   OR trim(project_type_name) = ''
   OR lower(trim(project_type_name)) = 'unknown';

##############################################
-- check_name: project_status_is_new_value
SELECT COUNT(*) AS violation_count
FROM de_prd.gold.wkdy_dim_project
WHERE project_status IS NOT NULL
  AND lower(trim(project_status)) NOT IN (
    'active','closed','pending close','schedule pending','suspended'
  );

##############################################
-- check_name: project_start_after_end_date
SELECT COUNT(*) AS violation_count
FROM de_prd.gold.wkdy_dim_project
WHERE coalesce(project_start_date, date '1900-01-01')
    > coalesce(project_end_date,   date '9999-12-31');

In [0]:
from pyspark.sql import functions as F

# pick singular vs plural, depending on your DQX version
errors_col = "_errors" if "_errors" in df_filtered.columns else "_error"

# array<struct<column:string,value:string>> for all non-system cols
base_cols = sorted([c for c in df_filtered.columns if not c.startswith("_")])
row_kv = F.array(*[F.struct(F.lit(c).alias("column"), F.col(c).cast("string").alias("value")) for c in base_cols])

# stable fingerprints (JSON of ordered arrays → sha2)
row_kv_fp = F.sha2(F.to_json(row_kv), 256)

# normalize errors for stable hashing (exclude volatile fields like run_time/user_metadata)
err_norm = F.transform(
    F.col(errors_col),
    lambda r: F.struct(
        r["name"].alias("name"),
        r["message"].alias("message"),
        F.array_sort(r["columns"]).alias("columns"),
        r["filter"].alias("filter"),
        r["function"].alias("function"),
    ),
)
errors_fp = F.sha2(F.to_json(F.array_sort(err_norm)), 256)

df_out = df_filtered.select(
    F.col(errors_col).alias("_errors"),
    row_kv.alias("row_kv"),
    row_kv_fp.alias("row_kv_fp"),
    errors_fp.alias("errors_fp"),
)

display(df_out)

#### Validation

In [0]:
# === Inline DQX checks (from your wkdy_dim_project.yaml), run them, and keep ONLY errors/warnings columns added ===
from collections import defaultdict
import yaml
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from pyspark.sql import functions as F

# ---- 1) Define checks inline (YAML in this cell) ----
checks_yaml = """
- table_name: de_prd.gold.wkdy_dim_project
  name: project_key_is_not_unique
  criticality: error
  run_config_name: default
  check:
    function: is_unique
    arguments:
      columns: [project_key]

- table_name: de_prd.gold.wkdy_dim_project
  name: project_key_is_null
  criticality: error
  run_config_name: default
  check:
    function: is_not_null
    arguments:
      column: project_key

- table_name: de_prd.gold.wkdy_dim_project
  name: project_status_is_null
  criticality: error
  run_config_name: default
  check:
    function: is_not_null
    arguments:
      column: project_status

- table_name: de_prd.gold.wkdy_dim_project
  name: project_status_is_new_value
  criticality: error
  run_config_name: default
  check:
    function: is_in_list
    arguments:
      column: project_status
      allowed:
        - Active
        - Closed
        - Pending Close
        - Schedule Pending
        - Suspended

- table_name: de_prd.gold.wkdy_dim_project
  name: project_type_is_not_null_or_unknown
  criticality: error
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "project_type_name is not null AND project_type_name != 'Unknown'"

- table_name: de_prd.gold.wkdy_dim_project
  name: any_email_has_invalid_format
  criticality: error
  run_config_name: default
  check:
    function: regex_match
    for_each_column:
      - project_leader_email
      - project_biller_email
      - customer_leader_email
      - customer_biller_email
    arguments:
      regex: "^(.+)@(.+)$"

- table_name: de_prd.gold.wkdy_dim_project
  name: project_start_after_end_date
  criticality: error
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(project_start_date, '1900-01-01') <= coalesce(project_end_date, '9999-12-31')"

- table_name: de_prd.gold.wkdy_dim_project
  name: first_activity_date_after_last_activity_date
  criticality: error
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(first_activity_date, '1900-01-01') <= coalesce(last_activity_date, '9999-12-31')"

- table_name: de_prd.gold.wkdy_dim_project
  name: created_date_after_last_updated_date
  criticality: error
  run_config_name: default
  check:
    function: sql_expression
    arguments:
      expression: "coalesce(_created_date, '1900-01-01') <= coalesce(_last_updated_date, '9999-12-31')"
"""

checks = yaml.safe_load(checks_yaml)
if isinstance(checks, dict):
    checks = [checks]

# Group rules by table
by_table = defaultdict(list)
for r in checks:
    t = r.get("table_name")
    if t:
        by_table[t].append(r)

print(f"Loaded {sum(len(v) for v in by_table.values())} rule(s) "
      f"across {len(by_table)} table(s): {list(by_table.keys())}")

# ---- 2) Validate & apply ----
dq = DQEngine(WorkspaceClient())

# Validate once (will raise if bad)
status = dq.validate_checks(checks)
if getattr(status, "has_errors", False):
    # if you prefer non-fatal, comment the raise and print status.to_string()
    raise ValueError(status.to_string())

for table, tbl_rules in by_table.items():
    print(f"\n=== Applying {len(tbl_rules)} rule(s) to {table} ===")
    df = spark.table(table)

    annotated = dq.apply_checks_by_metadata(df, tbl_rules)

    # DQX may use _error/_warning or _errors/_warnings depending on version
    err_col = "_errors"  if "_errors"  in annotated.columns else "_error"
    wrn_col = "_warnings" if "_warnings" in annotated.columns else "_warning"

    # Add normalized columns named 'errors' and 'warnings' (keep originals too)
    annotated = (annotated
                 .withColumn("errors",   F.col(err_col))
                 .withColumn("warnings", F.col(wrn_col)))

    # Simple summary (no RDD usage)
    agg = (
        annotated
        .select(
            F.size(F.col(err_col)).alias("e_sz"),
            F.size(F.col(wrn_col)).alias("w_sz"),
        )
        .agg(
            F.coalesce(F.sum("e_sz"), F.lit(0)).alias("errors"),
            F.coalesce(F.sum("w_sz"), F.lit(0)).alias("warnings"),
        )
        .collect()[0]
    )
    print(f"Hit summary -> errors={int(agg['errors'])}, warnings={int(agg['warnings'])}")

    # Show only rows that have any hits, with the new columns
    hits_only = annotated.filter(
        (F.col(err_col).isNotNull() & (F.size(F.col(err_col)) > 0)) |
        (F.col(wrn_col).isNotNull() & (F.size(F.col(wrn_col)) > 0))
    )

    try:
        display(hits_only.select("*"))   # notebook
    except NameError:
        hits_only.select("*").show(20, truncate=False)

    # OPTIONAL: persist annotated rows (with errors/warnings) to a Delta table
    # Uncomment and set a destination if you want to save:
    # results_table = "dq_dev.dqx.wkdy_dim_project_results"
    # (hits_only
    #   .select("*")   # includes 'errors' and 'warnings'
    #   .write.format("delta").mode("overwrite").saveAsTable(results_table))
    # print(f"Wrote {results_table}")